### MAIN WORK

The purpose of this a half hour coding session is to make a simple observation that can be noticed when rotating the digit 1 (MNIST) and its effect on the uncertainty. Ref: https://arxiv.org/pdf/1806.01768.pdf

Feel free to substitute this idea by something more challenging / different application

![rotating](rotating_page-0001.jpg)

![rotating.pdf](cls_probs__page-0001.jpg)

![rotating.pdf](rotating_dir_page-0001.jpg)

![rotating.pdf](cls_probs_dir_page-0001.jpg)

Let's go!

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.ndimage as nd
import scipy
import torch

In [2]:
from torchvision.datasets import mnist
from torch import nn
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from torch.optim import SGD
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Compose
from torchvision.utils import make_grid
from torch.utils.data.sampler import SubsetRandomSampler

device = 'cuda' if torch.cuda.is_available() else 'cpu'

Step 1: load mnist (normalized [0,1], with one hot encoding)

In [ ]:
# your code

Step 1b. Plot one of the image from the dataset

In [ ]:
# your code

Step 1c. Split into train and validation

In [10]:
# your code

Step 2. Define your Lenet-5 model

In [12]:
# your code

Step 3. Define training objective (CE / MSE) and way to compute accuracy

In [16]:
# your code

Step 4. Initialize your model and optimizer

In [26]:
# your code

Step 5. Write train loop

In [ ]:
# your code

Step 5b. Execute it and plot train/test loss values

In [ ]:
# your code

Step 6. Take benefit of

In [81]:
n_classes = 10
mnist_size = 28
softmax = nn.Softmax()

def rotate_img(x, deg):
    return nd.rotate(x.reshape(mnist_size,mnist_size),deg,reshape=False).ravel()


def rotating_image_classification(img, model, uncertainty=None, threshold=0.5):
    Mdeg = 180 
    Ndeg = int(Mdeg/10)+1
    ldeg = []
    lp = []
    lu=[]
    scores = np.zeros((1,n_classes))
    rimgs = np.zeros((mnist_size,mnist_size*Ndeg))
    for i,deg in enumerate(np.linspace(0,Mdeg, Ndeg)):
        nimg = rotate_img(img,deg).reshape(mnist_size,mnist_size)
        nimg = np.clip(a=nimg,a_min=0,a_max=1)
        rimgs[:,i*mnist_size:(i+1)*mnist_size] = nimg
        single_input = torch.reshape(torch.tensor(nimg), (1, 1, mnist_size, mnist_size))
        p_pred_t = softmax(model(single_input)).detach().numpy()
        scores += (p_pred_t >= threshold)
        ldeg.append(deg) 
        lp.append(p_pred_t[0])
        if uncertainty is not None:
            # TODO:
            u = None
            lu.append(u)
        scores += p_pred_t >= threshold

    labels = np.arange(10)[scores[0].astype(bool)]
    lp = np.array(lp)[:,labels]
    c = ['black','blue','red','brown','purple','cyan']
    marker = ['s','^','o']*2
    labels = labels.tolist()
    for i in range(len(labels)):
        plt.plot(ldeg,lp[:,i],marker=marker[i],c=c[i])

    if uncertainty is not None:
        labels += ['uncertainty']
        plt.plot(ldeg,lu,marker='<',c='red')
        
    plt.legend(labels)

    plt.xlim([0,Mdeg])
    plt.xlabel('Rotation Degree')
    plt.ylabel('Classification Probability')
    plt.show()

    plt.figure(figsize=[6.2,100])
    plt.imshow(1-rimgs,cmap='gray')
    plt.axis('off')
    plt.show()


Step 7. Expected Mean-Squared Error (get familiarized with the paper)

As described in the paper, a neural network can be trained to learn parameters of a Dirichlet distribution, instead of softmax probabilities. Dirichlet distributions with parameters $\alpha \geq 1$ behaves like a generative model for softmax probabilities (categorical distributions). It associates a likelihood value with each categorical distribution.

In [ ]:
# your code

Step 8. Off you go! you know what to do!

In [ ]:
# your code